The script
- recalculates implied volatilites in Julia based on the results from MATLAB
- calculates moneyness measures

## Implied volatility

- re-calculate implied volatilities in Julia and join them with dates and option IDs

In [1]:
@time include("../startup_script.jl")

elapsed time: 0.521153522 seconds (52396148 bytes allocated, 7.33% gc time)
elapsed time: 52.980657877 seconds (5419744236 bytes allocated, 65.55% gc time)
elapsed time: 0.728349379 seconds (112671656 bytes allocated, 49.54% gc time)
elapsed time: 67.373304003 seconds (5419744236 bytes allocated, 72.28% gc time)
elapsed time: 143.792628095 seconds (14793920072 bytes allocated, 60.31% gc time)


,Date,ID,Bid,Ask,Volume,Open_Interest
1,2006-07-03,c_20061215_1800,NA,NA,1,104
2,2006-07-03,p_20061215_1800,NA,NA,0,5515
3,2006-07-03,c_20061215_2000,NA,NA,0,2152
4,2006-07-03,p_20061215_2000,NA,NA,0,20941
5,2006-07-03,c_20061215_2200,NA,NA,0,2
6,2006-07-03,p_20061215_2200,NA,NA,0,4626


In [2]:
using Econometrics

- load implied volatilites calculated in Matlab and attach metadata

In [3]:
implVolasMatlab = readtable("../data/rel_data/implVolas_matlab.csv", header = false, nastrings = ["NaN"])
nObs = size(implVolasMatlab, 1)

iv = optPrices[[:Date, :ID]]
iv[:IV_Matlab] = implVolasMatlab[:x1]

head(iv)

,Date,ID,IV_Matlab
1,2006-07-03,c_20061215_1800,NA
2,2006-07-03,p_20061215_1800,0.50594
3,2006-07-03,c_20061215_2000,NA
4,2006-07-03,p_20061215_2000,0.46166
5,2006-07-03,c_20061215_2200,NA
6,2006-07-03,p_20061215_2200,0.42165


- get all data in one table which is required for the calculation of implied volatilities

In [4]:
ivData = join(optPrices, opts, on = :ID) |>
x -> join(x, cohortParams, on = [:Date, :Expiry]) |>
x -> join(x, daxVals, on = :Date) |>
x -> join(x, iv, on = [:Date, :ID])

head(ivData)

,Date,ID,Price,Expiry,Strike,IsCall,EONIA_matched,Time_to_Maturity,DAX,IV_Matlab
1,2006-07-03,c_20060721_4500,1212.0,2006-07-21,4500,true,0.0283102283088403,0.0549019607843137,5712.69,NA
2,2006-07-04,c_20060721_4500,1232.6,2006-07-21,4500,true,0.0282186257343229,0.0509803921568627,5729.01,NA
3,2006-07-05,c_20060721_4500,1131.1,2006-07-21,4500,true,0.0281095973321041,0.0470588235294118,5625.63,NA
4,2006-07-06,c_20060721_4500,1197.0,2006-07-21,4500,true,0.0281618489048387,0.0431372549019608,5695.47,NA
5,2006-07-07,c_20060721_4500,1185.1,2006-07-21,4500,true,0.0281130672686915,0.0392156862745098,5681.85,NA
6,2006-07-10,c_20060721_4500,1210.3,2006-07-21,4500,true,0.0281756309615022,0.0352941176470588,5706.32,NA


- define function to extract information required to calculate Black-Scholes prices

In [5]:
bsPriceInputExtractor = x -> (x[1, :DAX], x[1, :Strike], x[1, :EONIA_matched], x[1, :Time_to_Maturity])

(anonymous function)

- in some cases the observed option price is so low that there simply is no volatility that could cause such a low option price 
- for example, even with volatility almost zero the first option would be still more worth in a Black-Scholes world than the real market price

In [6]:
(ivData[1, :Price], bsCall(0.0000000001, bsPriceInputExtractor(ivData[1, :])...))

(1212.0,1219.678858946394)

- for such cases, no implied volatility can be found
- find cases where no implied volatility is possible

In [7]:
counter = 0
otherInds = falses(nObs)
currPrice = 0.
for ii=1:nObs
    if isna(ivData[ii, :IV_Matlab])
        if ivData[ii, :IsCall]
            currPrice = bsCall(0.0000000001, bsPriceInputExtractor(ivData[ii, :])...)
        else
            currPrice = bsPut(0.0000000001, bsPriceInputExtractor(ivData[ii, :])...)
        end
        if currPrice > ivData[ii, :Price]
            counter += 1
        else
            otherInds[ii] = true
        end
    end
end

counter / sum(isna(ivData[:IV_Matlab]))

0.9993888235429784

- hence, this seems to be the reason in almost all cases of `NA` values for implied volatility

- in other cases, however, the implied volatility would simply need to be higher than Matlab's default volatility limit

In [8]:
ivNoValue = ivData[otherInds, :]

,Date,ID,Price,Expiry,Strike,IsCall,EONIA_matched,Time_to_Maturity,DAX,IV_Matlab
1,2009-03-19,c_20090320_500,3545.4,2009-03-20,500,true,0.00926692907052475,0.00392156862745098,4043.46,NA
2,2009-06-18,c_20090619_1000,3840.9,2009-06-19,1000,true,0.00888045180593722,0.00392156862745098,4837.48,NA
3,2009-06-17,c_20090619_500,4307.797,2009-06-19,500,true,0.0089597413714718,0.00784313725490196,4799.98,NA
4,2009-06-18,c_20090619_500,4340.898,2009-06-19,500,true,0.00888045180593722,0.00392156862745098,4837.48,NA
5,2009-09-17,c_20090918_1000,4737.5,2009-09-18,1000,true,0.00339423306801562,0.00392156862745098,5731.14,NA
6,2009-09-17,c_20090918_500,5237.5,2009-09-18,500,true,0.00339423306801562,0.00392156862745098,5731.14,NA
7,2009-12-17,c_20091218_1000,4856.5,2009-12-18,1000,true,0.00376291136052792,0.00392156862745098,5844.44,NA
8,2009-12-17,c_20091218_1100,4756.5,2009-12-18,1100,true,0.00376291136052792,0.00392156862745098,5844.44,NA
9,2009-12-17,c_20091218_1200,4656.5,2009-12-18,1200,true,0.00376291136052792,0.00392156862745098,5844.44,NA
10,2009-12-17,c_20091218_1300,4556.5,2009-12-18,1300,true,0.00376291136052792,0.00392156862745098,5844.44,NA


- for example, an implied volatility of approximately 14 seems to do the job for the first option here

In [9]:
(ivNoValue[1, :Price], bsCall(14., bsPriceInputExtractor(ivNoValue[1, :])...))

(3545.4,3546.764104033321)

- all of these cases occur with maximum 3 days left to maturity

In [10]:
sum(ivNoValue[:Time_to_Maturity] .> 0.008)

2

In [11]:
sum(ivNoValue[:Time_to_Maturity] .> 0.012)

0

- hence, it might be worthwhile to set a filter that excludes options with short remaining time to expiry in any ongoing analysis

- the associated volumes are 0 almost everywhere

In [12]:
join(ivNoValue, addObs, on = [:Date, :ID])

,Date,ID,Price,Expiry,Strike,IsCall,EONIA_matched,Time_to_Maturity,DAX,IV_Matlab,Bid,Ask,Volume,Open_Interest
1,2009-03-19,c_20090320_500,3545.4,2009-03-20,500,true,0.00926692907052475,0.00392156862745098,4043.46,NA,NA,NA,0,30
2,2009-06-18,c_20090619_1000,3840.9,2009-06-19,1000,true,0.00888045180593722,0.00392156862745098,4837.48,NA,NA,NA,0,1650
3,2009-06-17,c_20090619_500,4307.797,2009-06-19,500,true,0.0089597413714718,0.00784313725490196,4799.98,NA,NA,NA,0,270
4,2009-06-18,c_20090619_500,4340.898,2009-06-19,500,true,0.00888045180593722,0.00392156862745098,4837.48,NA,NA,NA,0,270
5,2009-09-17,c_20090918_1000,4737.5,2009-09-18,1000,true,0.00339423306801562,0.00392156862745098,5731.14,NA,NA,NA,0,1020
6,2009-09-17,c_20090918_500,5237.5,2009-09-18,500,true,0.00339423306801562,0.00392156862745098,5731.14,NA,NA,NA,0,267
7,2009-12-17,c_20091218_1000,4856.5,2009-12-18,1000,true,0.00376291136052792,0.00392156862745098,5844.44,NA,NA,NA,0,627
8,2009-12-17,c_20091218_1100,4756.5,2009-12-18,1100,true,0.00376291136052792,0.00392156862745098,5844.44,NA,NA,NA,0,14
9,2009-12-17,c_20091218_1200,4656.5,2009-12-18,1200,true,0.00376291136052792,0.00392156862745098,5844.44,NA,NA,NA,750,650
10,2009-12-17,c_20091218_1300,4556.5,2009-12-18,1300,true,0.00376291136052792,0.00392156862745098,5844.44,NA,NA,NA,0,220


### Recalculate implied volatilities

- make sure that all metadata was correctly assigned
- extend implied volatility search to larger values for cases with short remaining time to expiry

In [13]:
head(ivData)

,Date,ID,Price,Expiry,Strike,IsCall,EONIA_matched,Time_to_Maturity,DAX,IV_Matlab
1,2006-07-03,c_20060721_4500,1212.0,2006-07-21,4500,true,0.0283102283088403,0.0549019607843137,5712.69,NA
2,2006-07-04,c_20060721_4500,1232.6,2006-07-21,4500,true,0.0282186257343229,0.0509803921568627,5729.01,NA
3,2006-07-05,c_20060721_4500,1131.1,2006-07-21,4500,true,0.0281095973321041,0.0470588235294118,5625.63,NA
4,2006-07-06,c_20060721_4500,1197.0,2006-07-21,4500,true,0.0281618489048387,0.0431372549019608,5695.47,NA
5,2006-07-07,c_20060721_4500,1185.1,2006-07-21,4500,true,0.0281130672686915,0.0392156862745098,5681.85,NA
6,2006-07-10,c_20060721_4500,1210.3,2006-07-21,4500,true,0.0281756309615022,0.0352941176470588,5706.32,NA


- define function to extract data required to calculate implied volatilities

In [14]:
ivInputExtractor = x -> (x[1, :Price], x[1, :DAX], x[1, :Strike], x[1, :EONIA_matched], x[1, :Time_to_Maturity])

(anonymous function)

- check extractor function:

In [15]:
ivInputExtractor(ivData[1, :])

(1212.0,5712.69,4500,0.0283102283088403,0.0549019607843137)

- try to improve values by calculating implied volatilites with Matlab values as starting values

In [16]:
prec = 0.0000001

implVolasJulia = DataArray(Float64, nObs)

@time begin
    for ii=1:nObs
        if !isna(ivData[ii, :IV_Matlab])
            inputs = ivInputExtractor(ivData[ii, :])
            implVolasJulia[ii] = implVola(ivData[ii, :IV_Matlab], inputs..., prec, ivData[ii, :IsCall])[1]
        end
    end
end

elapsed time: 172.642675109 seconds (14599282888 bytes allocated, 82.12% gc time)


- this did not produce new NAs

In [17]:
sum(isna(implVolasJulia) & !(isna(ivData[:IV_Matlab])))

0

In [18]:
(sum(isna(implVolasJulia)), sum(isna(ivData[:IV_Matlab])))

(86718,86718)

- hence, all Julia calculated implied volatilities seem to be valid values
- check, whether Julia implied volatilities did give an improvement, and whether the required precision is fulfilled for all values

In [19]:
improvement = falses(nObs)
bestIV = DataArray(Float64, nObs)
requPrec = falses(nObs)

for ii=1:nObs
    c1 = 0.
    c2 = 0.
    
    if !isna(ivData[ii, :IV_Matlab])
    
        if ivData[ii, :IsCall]
            # matlab price and julia price
            c1 = bsCall(ivData[ii, :IV_Matlab], bsPriceInputExtractor(ivData[ii, :])...)
            c2 = bsCall(implVolasJulia[ii], bsPriceInputExtractor(ivData[ii, :])...)
        else
            # matlab price and julia price
            c1 = bsPut(ivData[ii, :IV_Matlab], bsPriceInputExtractor(ivData[ii, :])...)
            c2 = bsPut(implVolasJulia[ii], bsPriceInputExtractor(ivData[ii, :])...)
        end
        
        diff1 = abs(ivData[ii, :Price] - c1)
        diff2 = abs(ivData[ii, :Price] - c2)
        if diff2 <= diff1
            improvement[ii] = true
            bestIV[ii] = implVolasJulia[ii]
            if diff2 < prec
                requPrec[ii] = true
            end
        else
            # no improvement
            bestIV[ii] = ivData[ii, :IV_Matlab]
            if diff1 < prec
                requPrec[ii] = true
            end
        end
    end
end

- is required precision fulfilled for all cases without `NA`?

In [20]:
all(requPrec[!isna(ivData[:IV_Matlab])])

true

- how large is the fraction of improved (or equally good) implied values?

In [21]:
sum(improvement[!isna(ivData[:IV_Matlab])])/length(improvement[!isna(ivData[:IV_Matlab])])

1.0

- how many cases have not improved?

In [22]:
length(improvement[!isna(ivData[:IV_Matlab])]) - sum(improvement[!isna(ivData[:IV_Matlab])])

0

#### Get implied volatility for cases exceeding Matlab's default implied volatility limit

- find cases where vola is still possible

In [23]:
possibleInds = falses(nObs)
for ii=1:nObs
    if isna(ivData[ii, :IV_Matlab])
        if ivData[ii, :IsCall]
            # if NA is not produced because BS price is always too high
            if ivData[ii, :Price] > bsCall(0.0000000001, bsPriceInputExtractor(ivData[ii, :])...)
                possibleInds[ii] = true
            end
        else
            if ivData[ii, :Price] > bsPut(0.0000000001, bsPriceInputExtractor(ivData[ii, :])...)
                possibleInds[ii] = true
            end
        end
    end
end

- number of volatilities that still can be calculated

In [24]:
nToTry = sum(possibleInds)

53

- show cases

In [25]:
ivData[possibleInds, :]

,Date,ID,Price,Expiry,Strike,IsCall,EONIA_matched,Time_to_Maturity,DAX,IV_Matlab
1,2009-03-19,c_20090320_500,3545.4,2009-03-20,500,true,0.00926692907052475,0.00392156862745098,4043.46,NA
2,2009-06-18,c_20090619_1000,3840.9,2009-06-19,1000,true,0.00888045180593722,0.00392156862745098,4837.48,NA
3,2009-06-17,c_20090619_500,4307.797,2009-06-19,500,true,0.0089597413714718,0.00784313725490196,4799.98,NA
4,2009-06-18,c_20090619_500,4340.898,2009-06-19,500,true,0.00888045180593722,0.00392156862745098,4837.48,NA
5,2009-09-17,c_20090918_1000,4737.5,2009-09-18,1000,true,0.00339423306801562,0.00392156862745098,5731.14,NA
6,2009-09-17,c_20090918_500,5237.5,2009-09-18,500,true,0.00339423306801562,0.00392156862745098,5731.14,NA
7,2009-12-17,c_20091218_1000,4856.5,2009-12-18,1000,true,0.00376291136052792,0.00392156862745098,5844.44,NA
8,2009-12-17,c_20091218_1100,4756.5,2009-12-18,1100,true,0.00376291136052792,0.00392156862745098,5844.44,NA
9,2009-12-17,c_20091218_1200,4656.5,2009-12-18,1200,true,0.00376291136052792,0.00392156862745098,5844.44,NA
10,2009-12-17,c_20091218_1300,4556.5,2009-12-18,1300,true,0.00376291136052792,0.00392156862745098,5844.44,NA


- try to find improvement for these cases

In [26]:
# specify high initial implied vola levels
sigmaHigh = [8., 10, 15, 20, 25, 30, 35, 40, 60, 80, 100]

highVolas = Array(Float64, nToTry, length(sigmaHigh))
pDiffs = Array(Float64, nToTry, length(sigmaHigh))

rowCounter = 1

for ii=1:nObs
    if possibleInds[ii]
        inputs = ivInputExtractor(ivData[ii, :])
        for kk=1:length(sigmaHigh)
            ivRes = implVola(sigmaHigh[kk], inputs..., prec, ivData[ii, :IsCall])
            if !isna(ivRes[1])
                highVolas[rowCounter, kk] = ivRes[1]
            else
                highVolas[rowCounter, kk] = NaN
            end
            if !isna(ivRes[2])
                pDiffs[rowCounter, kk] = ivRes[2]
            else
                pDiffs[rowCounter, kk] = NaN
            end
        end
        # find best volatility
        minDiffInd = indmin(pDiffs[rowCounter, :])
        bestIVval = highVolas[rowCounter, minDiffInd]
        
        # write to bestIV
        bestIV[ii] = bestIVval
        rowCounter += 1
    end
end
        
highVolas

53x11 Array{Float64,2}:
  Inf         13.1358  13.1358  …  13.1358    13.1358   Inf  -Inf  Inf
   10.3562    10.3562  10.3562     10.3562    10.3562   Inf  -Inf  Inf
   11.7865    11.7865  11.7865     11.7865  -Inf       -Inf   Inf  Inf
 -Inf       -Inf       14.9703     14.9703    14.9703  -Inf  -Inf  Inf
  Inf         12.14    12.14       12.14      12.14     Inf  -Inf  Inf
 -Inf        Inf       17.1175  …  17.1175    17.1175  -Inf  -Inf  Inf
  Inf         13.3907  13.3907     13.3907    13.3907   Inf  -Inf  Inf
  Inf         12.6685  12.6685     12.6685    12.6685   Inf  -Inf  Inf
   12.0148    12.0148  12.0148     12.0148    12.0148   Inf  -Inf  Inf
   11.4179    11.4179  11.4179     11.4179    11.4179   Inf  -Inf  Inf
   10.3602    10.3602  10.3602  …  10.3602    10.3602   Inf  -Inf  Inf
 -Inf        Inf       18.8577     18.8577    18.8577  -Inf  -Inf  Inf
 -Inf        Inf       17.369      17.369     17.369   -Inf  -Inf  Inf
    ⋮                           ⋱                    

- show improved values

In [27]:
bestIV[possibleInds]

53-element DataArray{Float64,1}:
 13.1358
 10.3562
 11.7865
 14.9703
 12.14  
 17.1175
 13.3907
 12.6685
 12.0148
 11.4179
 10.3602
 18.8577
 17.369 
  ⋮     
 10.71  
 10.0462
 11.5386
 10.6661
 12.3153
 11.4601
 10.5872
 11.7342
 12.4663
 10.3938
 12.7416
 10.1333

- attach metadata

In [28]:
impliedVolas = ivData[[:Date, :ID]]
impliedVolas[:IV] = bestIV

head(impliedVolas)

,Date,ID,IV
1,2006-07-03,c_20060721_4500,NA
2,2006-07-04,c_20060721_4500,NA
3,2006-07-05,c_20060721_4500,NA
4,2006-07-06,c_20060721_4500,NA
5,2006-07-07,c_20060721_4500,NA
6,2006-07-10,c_20060721_4500,NA


- write to disk

In [29]:
writetable("../data/rel_data/implVola.csv", impliedVolas)

### Calculate moneyness measures

In [30]:
mnynessData = join(optPrices, opts, on = :ID) |>
x -> join(x, cohortParams, on = [:Date, :Expiry]) |>
x -> join(x, daxVals, on = [:Date])

head(mnynessData)

,Date,ID,Price,Expiry,Strike,IsCall,EONIA_matched,Time_to_Maturity,DAX
1,2006-07-03,c_20060721_4500,1212.0,2006-07-21,4500,true,0.0283102283088403,0.0549019607843137,5712.69
2,2006-07-03,c_20060721_4600,1112.3,2006-07-21,4600,true,0.0283102283088403,0.0549019607843137,5712.69
3,2006-07-03,c_20060721_4700,1012.7,2006-07-21,4700,true,0.0283102283088403,0.0549019607843137,5712.69
4,2006-07-03,c_20060721_4800,913.2,2006-07-21,4800,true,0.0283102283088403,0.0549019607843137,5712.69
5,2006-07-03,c_20060721_4850,863.5,2006-07-21,4850,true,0.0283102283088403,0.0549019607843137,5712.69
6,2006-07-03,c_20060721_4900,813.9,2006-07-21,4900,true,0.0283102283088403,0.0549019607843137,5712.69


In [31]:
absMny = zeros(Float64, nObs)
simpleMny = zeros(Float64, nObs)
invMny = zeros(Float64, nObs)
logSimpleMny = zeros(Float64, nObs)
logForwardMny = zeros(Float64, nObs)
sclMny = zeros(Float64, nObs)

for ii=1:nObs
    # absolute moneyness
    absMny[ii] = mnynessData[ii, :DAX] - mnynessData[ii, :Strike]
    
    # relative moneyness
    simpleMny[ii] = mnynessData[ii, :DAX]/mnynessData[ii, :Strike]
    
    # inverse relative moneyness
    invMny[ii] = mnynessData[ii, :Strike]/mnynessData[ii, :DAX]
    
    # log simple moneyness
    logSimpleMny[ii] = log(mnynessData[ii, :DAX]/mnynessData[ii, :Strike])
    
    # log forward moneyness
    logForwardMny[ii] = log(mnynessData[ii, :DAX]/mnynessData[ii, :Strike]) + mnynessData[ii, :EONIA_matched]*mnynessData[ii, :Time_to_Maturity]
    
    # time scaled moneyness
    sclMny[ii] = log(mnynessData[ii, :DAX]/mnynessData[ii, :Strike])/sqrt(mnynessData[ii, :Time_to_Maturity])
end


In [32]:
mnyness = mnynessData[[:Date, :ID]]
mnyness[:AbsMny] = absMny
mnyness[:SimpMny] = simpleMny
mnyness[:InvMny] = invMny
mnyness[:LogMny] = logSimpleMny
mnyness[:LogFMny] = logForwardMny
mnyness[:SclMny] = sclMny

head(mnyness)

,Date,ID,AbsMny,SimpMny,InvMny,LogMny,LogFMny,SclMny
1,2006-07-03,c_20060721_4500,1212.6899999999996,1.2694866666666667,0.7877199708018465,0.23861261928480745,0.24016690632921436,1.0183556574518589
2,2006-07-03,c_20060721_4600,1112.6899999999996,1.2418891304347826,0.8052248590418875,0.21663371256603217,0.21818799961043908,0.9245536445124007
3,2006-07-03,c_20060721_4700,1012.6899999999996,1.2154659574468085,0.8227297472819285,0.19512750734506865,0.19668179438947556,0.8327690363775394
4,2006-07-03,c_20060721_4800,912.6899999999996,1.1901437499999998,0.8402346355219695,0.17407409814723615,0.17562838519164306,0.7429168800686095
5,2006-07-03,c_20060721_4850,862.6899999999996,1.1778742268041236,0.84898707964199,0.1637113111116896,0.1652655981560965,0.6986903725341459
6,2006-07-03,c_20060721_4900,812.6899999999996,1.1658551020408163,0.8577395237620106,0.1534548109445006,0.15500909798890752,0.6549174781993078


In [33]:
writetable("../data/rel_data/mnyness.csv", mnyness)

### Sessioninfo

In [34]:
versioninfo()

Julia Version 0.3.6
Commit a05f87b* (2015-01-08 22:33 UTC)
Platform Info:
  System: Linux (x86_64-linux-gnu)
  CPU: Intel(R) Core(TM) i3-3240 CPU @ 3.40GHz
  WORD_SIZE: 64
  BLAS: libopenblas (DYNAMIC_ARCH NO_AFFINITY Sandybridge)
  LAPACK: libopenblas
  LIBM: libopenlibm
  LLVM: libLLVM-3.3


In [35]:
Pkg.status()

20 required packages:
 - DataArrays                    0.2.15
 - DataFrames                    0.6.6
 - Dates                         0.3.2
 - Debug                         0.1.3
 - Distributions                 0.7.3
 - Docile                        0.5.3
 - GLM                           0.4.6
 - Gadfly                        0.3.12
 - IJulia                        0.2.5
 - JuMP                          0.9.1
 - Lexicon                       0.1.10
 - MAT                           0.2.12
 - NLopt                         0.2.1
 - Plotly                        0.0.3+             master
 - Quandl                        0.4.1
 - RDatasets                     0.1.2
 - Requires                      0.1.2              master
 - Taro                          0.1.4
 - TimeSeries                    0.5.9
 - Winston                       0.11.10
58 additional packages:
 - ArrayViews                    0.6.2
 - AssetMgmt                     0.0.0-             master (unregistered)
 - BinDeps     